In [1]:
import os
import time

import numpy as np
import cv2
import scipy
from pydub import AudioSegment
from sklearn.utils import shuffle
import queue
import os

import numpy as np
import librosa
from pathlib import Path
import pickle
import cv2
import pandas as pd

import numpy as np
import os
from scipy import  misc
from keras.models import model_from_json

/usr/local/lib/python3.6/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
Using TensorFlow backend.


In [2]:
os.listdir('birdsong-recognition/')

['spec_bandpass_11',
 '.~lock.example_test_audio_metadata.csv#',
 'noise',
 'spec_512_64_5sec_0sec',
 'noise_bandpass_512_128',
 '0044_XC256543_001.png',
 'train.csv',
 'example_test_audio',
 'sample_submission.csv',
 'spec_512_64',
 'test.csv',
 'noise_512_64',
 'labels.pkl',
 'spec_bandpass_512_128',
 'train_wavaudio',
 'del.py',
 '.~lock.example_test_audio_summary.csv#',
 'noise_512_64_5sec_0sec',
 'spec',
 'example_test_audio_metadata.csv',
 'spec_512_128_5sec_0sec',
 'noise_bandpass_11',
 'noise_512_128_5sec_0sec',
 'train',
 'example_test_audio_summary.csv']

In [3]:
#labels as Pickle file
classifier_f = open("int_to_word_spec_new_11july.pickle", "rb")
int_to_word_out = pickle.load(classifier_f)
classifier_f.close()

In [4]:
###############################################
def getRandomState():

    RANDOM_SEED = 1337
    RANDOM = np.random.RandomState(RANDOM_SEED)

    return RANDOM
######################## CONFIG #########################
RANDOM = getRandomState()

SAMPLE_RATE= 44100
SPEC_LENGTH= 5.0
SPEC_OVERLAP= 0.0
SPEC_MINLEN= 1.0
SPEC_FMIN= 500
SPEC_FMAX= 22000
SPEC_TYPE='melspec'

SPEC_SIGNAL_THRESHOLD= 0.0001
IM_DIM = 1
IM_SIZE = (512, 128)
BATCH_SIZE=1
NUM_CLASSES=264

In [6]:
def load_model():
    
    # load json and create model
    json_file = open('model_bird_spec_11thjuly.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("model_bird_specwithoutmeta.h5")
    print("Loaded model from disk")
    return loaded_model

model=load_model()

Loaded model from disk


In [7]:
from pathlib import Path
'''
TEST = Path("birdsong-recognition/test_audio").exists()

if TEST:
    DATA_DIR = "birdsong-recognition/"
else:
    # dataset created by @shonenkov, thanks!
    DATA_DIR = "birdcall-check/"
train_df = pd.read_csv('birdsong-recognition/train.csv')    
ebird_codes = np.array(sorted(list(set(train_df['ebird_code']))))
bird_to_idx = { bird: idx for idx, bird in enumerate(ebird_codes) }
'''
TEST_PATH = Path('birdsong-recognition') if os.path.exists('birdsong-recognition/test_audio') else Path('birdcall-check')

TEST_AUDIO_PATH = TEST_PATH/'test_audio'
test_df = pd.read_csv(TEST_PATH/'test.csv')
test_df.head()

,site,row_id,seconds,audio_id
0,site_1,site_1_b87d141f6dc24f69a51cdb5973fcd68a_5,5.0,b87d141f6dc24f69a51cdb5973fcd68a
1,site_1,site_1_b87d141f6dc24f69a51cdb5973fcd68a_10,10.0,b87d141f6dc24f69a51cdb5973fcd68a
2,site_1,site_1_b87d141f6dc24f69a51cdb5973fcd68a_15,15.0,b87d141f6dc24f69a51cdb5973fcd68a
3,site_1,site_1_b87d141f6dc24f69a51cdb5973fcd68a_20,20.0,b87d141f6dc24f69a51cdb5973fcd68a
4,site_1,site_1_b87d141f6dc24f69a51cdb5973fcd68a_25,25.0,b87d141f6dc24f69a51cdb5973fcd68a


In [8]:
def getSpecs(paths):
    
    specs = []
    #noise = []
    if row.site != 'site_3':
        print (row.site)
        end_seconds=int(row.seconds)
        start_second = end_seconds-5
        for spec in specsFromFile(paths,
                                    rate=SAMPLE_RATE,
                                    seconds=SPEC_LENGTH,
                                    overlap=SPEC_OVERLAP,
                                    minlen=SPEC_MINLEN,
                                    fmin=SPEC_FMIN,
                                    fmax=SPEC_FMAX,
                                    start=start_second,
                                    end=end_seconds,
                                    spec_type=SPEC_TYPE,
                                    shape=(IM_SIZE[1], IM_SIZE[0])):

        # Determine signal to noise ratio
        #s2n = signal2noise(spec)
             specs = spec.reshape(-1, IM_DIM, IM_SIZE[1], IM_SIZE[0])
    else:
        end_seconds=-1
        start_second = -1
        #SPEC_LENGTH=10.0
    # Get mel-specs for file
    #print (start_second)
    #print (end_seconds-start_second)
        for spec in specsFromFile(paths,
                                    rate=SAMPLE_RATE,
                                    seconds=SPEC_LENGTH,
                                    overlap=SPEC_OVERLAP,
                                    minlen=SPEC_MINLEN,
                                    fmin=SPEC_FMIN,
                                    fmax=SPEC_FMAX,
                                    start=start_second,
                                    end=end_seconds,
                                    spec_type=SPEC_TYPE,
                                    shape=(IM_SIZE[1], IM_SIZE[0])):

        # Determine signal to noise ratio
        #s2n = signal2noise(spec)
            spec = spec.reshape(-1, IM_DIM, IM_SIZE[1], IM_SIZE[0])
            specs.append(spec)
        #spec = spec.reshape(-1, IM_DIM, IM_SIZE[1], IM_SIZE[0])
        #print (len(specs))
        #noise.append(s2n)

    # Shuffle arrays (we want to select randomly later)
    #specs, noise = shuffle(specs, noise, random_state=RANDOM)

    return specs

In [9]:
def predictionPooling(p):
    
    #You can test different prediction pooling strategies here
    if p.ndim == 2:

        try:

            # Median filtered pooling for monophonic recordings
            row_median = np.median(p, axis=1, keepdims=True)
            p[p < row_median * 1.5] = 0.0
            p_pool = np.mean((p * 2) ** 2, axis=0)
            p_pool -= p_pool.min()
            if p_pool.max() > 1.0:
                p_pool /= p_pool.max()

            # Mean exponential pooling for monophonic recordings
            #p_pool = np.mean((p * 2) ** 2, axis=0)
            #p_pool[p_pool > 1.0] = 1.0

            # Simple average pooling
            #p_pool = np.mean(p, axis=0)
            #p_pool = sigmoid(p_pool)

        except:
            p_pool = cfg.getRandomState().normal(0.0, 1.0, (p.shape[1]))
        
    else:
        p_pool = p

    return p_pool

In [10]:
##################################################################
# -*- coding: utf-8 -*-
#Loading images with CPU background threads during GPU forward passes saves a lot of time
#Credit: J. Schlüter (https://github.com/Lasagne/Lasagne/issues/12)
def threadedBatchGenerator(generator, num_cached=24):
    
    #import Queue
    queues = queue.Queue(maxsize=num_cached)
    sentinel = object()  # guaranteed unique reference

    #define producer (putting items into queue)
    def producer():
        for item in generator:
            queues.put(item)
        queues.put(sentinel)

    #start producer (in a background thread)
    import threading
    thread = threading.Thread(target=producer)
    thread.daemon = True
    thread.start()

    #run as consumer (read items from queue, in current thread)
    item = queues.get()
    while item is not sentinel:
        yield item
        queues.task_done()
        item = queues.get()



In [11]:
%%time
row_id_birds = {}
spec_cnt = 0
for i in range(len(test_df)):


    row = test_df.iloc[i]
    audio_id = row.audio_id
    paths=TEST_AUDIO_PATH/(audio_id+'.mp3')
    if row.site != 'site_3':
        end_seconds=int(row.seconds)
        start_second = end_seconds-5

        

        spec = (getSpecs(paths))

        y=model.predict(spec)
       # print (np.max(y)) 
       # print (int_to_word_out[np.argmax(y)])
        if np.max(y)>=0.7:
            row_id_birds[row.row_id] = ''.join(int_to_word_out[np.argmax(y)])
        else: 
            row_id_birds[row.row_id] = 'nocall' 

            
    else:
        

        spec = (getSpecs(paths))
        predictions = []
        results = []
        for s in range(len(spec)):
               
                img = spec[s].reshape(-1, IM_DIM, IM_SIZE[1], IM_SIZE[0])

                y=model.predict(img)

               #print (np.max(y))
  
               
                if len(predictions):
                    predictions = np.vstack([predictions, y])  
                else:
                    predictions = y
        #print (int_to_word_out[np.argmax(predictions)])            

        for j in range(len(predictions)):
            
            #print (np.max(predictions[j])) 
            #print (int_to_word_out[np.argmax(predictions[j])])
            if np.max(predictions[j])>=0.7:
                    results.append(int_to_word_out[np.argmax(predictions[j])])
                    mylist = list(dict.fromkeys(results))



            else: 
                    
                    results.append('nocall')
                    mylist = list(dict.fromkeys(results))
                    
        if len(results)>1 and 'nocall' in mylist:
            mylist.remove('nocall')
            row_id_birds[row.row_id] = ','.join(mylist)        
        else:  
            row_id_birds[row.row_id] = ','.join(mylist)
            

         
        

site_1


/usr/local/lib/python3.6/dist-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


site_1


/usr/local/lib/python3.6/dist-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


site_1


...

site_2


/usr/local/lib/python3.6/dist-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


site_2


/usr/local/lib/python3.6/dist-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/usr/local/lib/python3.6/dist-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/usr/local/lib/python3.6/dist-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/usr/local/lib/python3.6/dist-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/usr/local/lib/python3.6/dist-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/usr/local/lib/python3.6/dist-packages/librosa/cor

CPU times: user 7min 37s, sys: 1min 9s, total: 8min 47s
Wall time: 2min 16s


In [14]:
row_id_birds
submission = pd.DataFrame({
    'row_id': test_df.row_id.values,
    'birds': [row_id_birds[row_id] for row_id in test_df.row_id.values]
})

In [15]:
submission.to_csv('sub.csv', index=False)
pd.set_option('display.max_rows', None)
submission

,row_id,birds
0,site_1_b87d141f6dc24f69a51cdb5973fcd68a_5,aldfly
1,site_1_b87d141f6dc24f69a51cdb5973fcd68a_10,nocall
2,site_1_b87d141f6dc24f69a51cdb5973fcd68a_15,aldfly
3,site_1_b87d141f6dc24f69a51cdb5973fcd68a_20,nocall
4,site_1_b87d141f6dc24f69a51cdb5973fcd68a_25,aldfly
5,site_1_8e2380fc2c1348c2b859fb624300cb36_5,aldfly
6,site_1_8e2380fc2c1348c2b859fb624300cb36_10,wilfly
7,site_1_8e2380fc2c1348c2b859fb624300cb36_15,nocall
8,site_1_8e2380fc2c1348c2b859fb624300cb36_20,aldfly
9,site_1_8e2380fc2c1348c2b859fb624300cb36_25,nocall
